In [ ]:
import pandas as pd
import csv

In [ ]:
data = pd.read_csv('../../data/u_watt_office.csv',sep=';')
data

In [ ]:
p1_u = []
p2_u = []
consumo = []
vol_u = []
hora = []
fecha = []
dias = 0
ts = []

watt_hora = 0
kw_hora = []
xhora = []
voltsHora = []
ampsHora = []
hora_volt = 0
hora_amp = 0
datosHora = []

watt_dia = 0
kw_dia = []
xdia = []

lista = list(data['ts'])
dia_ini = int(str(lista[0])[8:10])
hora_ini = int(str(lista[0])[11:13])

j=0
ini = 0


In [ ]:
horas_data = []
horas_count = []
hount = 0
timezone = 0 
for i in lista:
    # se obtienen los valores de sensores y voltaje 
    u_p1 = float(data[data['ts'] == i]['Sensor 1'].values)
    p1_u.append(u_p1)
    u_p2 = float(data[data['ts'] == i]['Sensor 2'].values)
    p2_u.append(u_p2)
    vol = float(data[data['ts'] == i]['voltaje'].values)
    vol_u.append(vol)
    # se calcula la potencia en kw
    carga = ((u_p1 + u_p2) * vol) /1000 
    consumo.append(carga)
    # se obtiene el tiempo y se separa por fecha y hora
    tiempo = str(data[data['ts'] == i]['ts'].values)
    #hora.append(tiempo.split("T")[1][:8])
    timezone = int(tiempo.split("T")[1][:2])
    if (timezone - 5) >= 0 :
        timezone = str(int(tiempo.split("T")[1][:2])-5)
        hora.append(timezone+tiempo.split("T")[1][2:8])
        fecha.append(tiempo.split("T")[0][2:])
        #print(timezone + tiempo.split("T")[1][2:8])
    else :
        timezone = str(int(tiempo.split("T")[1][:2])+19)
        hora.append(timezone+tiempo.split("T")[1][2:8])
        fecha.append(fecha[hount-1])
    # datos para utilizarlo a la hora
    #ts.append(tiempo)
    ts.append(fecha[j]+"T"+hora[j])
    watt_hora = watt_hora + carga
    hora_amp = hora_amp + (u_p1 + u_p2)
    hora_volt = hora_volt + vol
    hount = hount + 1
    if j+1 < len(lista):
        # pasado una hora
        if hora_ini != int(str(lista[j+1])[11:13]):
            kw_hora.append(watt_hora/hount)
            xhora.append(hora_ini)
            hora_ini = int(str(lista[j+1])[11:13])
            watt_dia = watt_dia + watt_hora/hount
            watt_hora = 0
            horas_data.append(i)
            horas_count.append(hount)
            ampsHora.append(hora_amp/hount)
            voltsHora.append(hora_volt/hount)
            datosHora.append(hount)
            hora_amp = 0
            hora_volt = 0
            hount = 0
        # pasado un dia
        if dia_ini != int(str(lista[j+1])[8:10]) and int(str(lista[j+1])[11:13]) == 0:
            kw_dia.append(watt_dia)
            xdia.append(dia_ini)
            watt_dia = 0
            #ini = j + 1
            dias = dias + 1
            dia_ini = int(str(lista[j+1])[8:10])
        j = j+1

dias


In [ ]:
h_data = pd.DataFrame({
    'ultimo dato' : horas_data,
    'datos' : horas_count
})
h_data.to_csv('./horas.csv')

In [ ]:
data_final = pd.DataFrame({
    'ts' : ts,
    'hora' :hora,
    'fecha': fecha,
    'S1' :p1_u,
    'S2' :p2_u,
    'Volts' : vol_u,
    'kW': consumo
    })

In [ ]:
data_xhoras = pd.DataFrame({
    'hora' : xhora,
    'kwh' : kw_hora,
    'Amps' : ampsHora,
    'volt' : voltsHora,
    'datos' : datosHora
})

In [ ]:
data_xdia = pd.DataFrame({
    'dia' : xdia,
    'consumo' : kw_dia
})

In [ ]:
data_final.to_csv('./final.csv')
data_xhoras.to_csv('./kw_hora.csv')
data_xdia.to_csv('./kw_dia.csv')